# Генералова Софья БКЛ184
## ДЗ 1
### Сначала загружаю модуль для проверки пепа и другие

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on
import json, re, string

### Укажите путь к файлу json (вида: C:\Users\Hey\Desktop\hw_3_twitter.json)

In [2]:
file = input('введите путь к файлу: ')

введите путь к файлу: C:\Users\Asus\Desktop\proga2year_dz1\hw_3_twitter.json


### Из файла по строкам добавляю в список, переводя по пути джейсон-строку в объекты питона.

In [3]:
twitter = []
for line in open(file, encoding='utf-8'):
    twitter.append(json.loads(line))

# задание 1
## Сколько твитов в наборе?
### для этого считаем сколько объектов лежит в списке twitter

In [4]:
tw_len = len(twitter)
print('ОТВЕТ:', tw_len)

ОТВЕТ: 2556


# задание 2
## Какой процент твитов составляют удаленные записи?
### прохожу по списку, прибавляю к счетчику +1, если встречаю key 'delete'

In [5]:
c = 0  # счетчик
for tw in twitter:
    if tw.get("delete"):
        c += 1

print('кол-во удаленных твитов:', c)

кол-во удаленных твитов: 362


### теперь считаю их процент от всех твитов

In [6]:
dolya_d = c / tw_len * 100

print('ОТВЕТ: доля удаленных твитов в наборе:', dolya_d)

ОТВЕТ: доля удаленных твитов в наборе: 14.162754303599373


In [7]:
from collections import Counter

# задание 3
## Какие самые популярные языки твитов? ТОП 15
### прохожу по списку с твитами, для каждого сохраняю язык в список. делаю частотный словарь  

In [8]:
# 3 самые популярные (15 sht) языки твитов?
langs = []
for tw in twitter:
    if tw.get('lang'):
        langs.append(tw.get('lang'))

langsort = Counter(langs)
# 15 most common langs
print('ОТВЕТ')
print('top 15 languages\n')
for key, val in langsort.most_common(15):
    print(key, val)

ОТВЕТ
top 15 languages

en 719
ja 438
es 173
ko 149
th 123
ar 119
und 117
in 71
pt 69
fr 35
tr 30
tl 29
hi 23
ru 15
fa 8


# задание 4
## Есть ли твиты от одного и того же пользователя? Если да, то сколько таких пользователей?
### у ретвитов с комментариями атрибут quoted_status, у просто ретвитов retweeted_status
### в этом задании я считаю авторов твитов из обоих видов ретвитов, а также из удаленных твитов, и, естественно, из твитов-не ретвитов

In [9]:
users = []
uniq_user = 0
for tw in twitter:
    if tw.get('retweeted_status'):
        # add orig tw author
        users.append(tw['retweeted_status']['user']['id_str'])
    elif tw.get('quoted_status'):
        # add orig tw author
        users.append(tw['quoted_status']['user']['id_str'])
    elif tw.get('user'):
        users.append(tw['user']['id_str'])  # для не ретвитов
    else:
        users.append(tw['delete']['status']['user_id_str'])  # авт из удал тв
usersort = Counter(users)
# перебираю пары ключ-значение в отсортированном по убыванию значения словаре
# прибавляю к счетчику 1, если значение больше 1
# как только встречается значение 1 можно останавливать перебор
for key, val in usersort.most_common():
    if val == 1:
        break
    else:
        uniq_user += 1
print('ОТВЕТ: кол-во пользователей: ', uniq_user)

ОТВЕТ: кол-во пользователей:  88


# задание 5
## Топ-20 хэштегов
### если есть атрибут энтитис, а в нем не пустой список хэштегов, то из каждого элемента в списке хэштегов по ключу 'text' достаю текст хэштега и добавляю в список. потом делаю частотный словарь

In [10]:
# 5 20 hashtags
hashtags = []
for tw in twitter:
    if tw.get('entities'):
        if len(tw['entities']['hashtags']) != 0:
            for tag in tw['entities']['hashtags']:
                hashtags.append(tag['text'])
hashtagsort = Counter(hashtags)
h = hashtagsort.most_common()
print('ОТВЕТ:')
print('top 20 hashtags:')
print()
for i in range(20):
    print(h[i][0], h[i][1])

ОТВЕТ:
top 20 hashtags:

BTS 17
방탄소년단 13
AMAs 11
人気投票ガチャ 8
태형 7
뷔 6
BTSinChicago 5
BTSLoveYourselfTour 5
오늘의방탄 5
PledgeForSwachhBharat 5
MPN 5
PCAs 4
V 4
시카고1회차공연 4
เป๊กผลิตโชค 4
JIMIN 4
running 3
NCT 3
지민 3
WajahmuPlastik 3


# задание 6
## Предобработать тексты оригинальных твитов (не ретвитов) на английском языке (убрать пунктуацию, привести к нижнему регистру) и составить частотный словарь
### прохожу по списку твитов, сначала проверяю, что язык англ
### потом проверяю, сокращен ли текст в атрибуте 'text', если да, то иду в 'full_text'
### если это ретвит, то пропускаю

In [11]:
# 6 частотный словарь слов в ориг твитах

origs = []
for tw in twitter:
    if tw.get('lang') == 'en':
        if tw.get('retweeted_status'):
            continue
        elif tw.get('quoted_status'):
            continue
        elif tw.get('delete'):
            continue
        else:
            if tw['truncated']:
                t = tw['extended_tweet']['full_text']
            else:
                t = tw['text']

        r = re.sub(r'http\S+', '', t)  # чистка от ссылок
        rr = re.sub(r'#[^\S]+', '', r)  # чистка от хэштегов
        # чистка от пунктуации
        res = re.sub("[" + re.escape(string.punctuation) + "]", "", rr)
        # понижение регистра
        res1 = res.lower()
        # делю на слова по пробельным символам и прибавляю их в список
        origs += res1.split()
origsort = Counter(origs)
print('ОТВЕТ:')
print('top 20 самых частотных слов:')
print()
for key, val in origsort.most_common(20):
    print(key, val)

ОТВЕТ:
top 20 самых частотных слов:

the 134
to 96
a 74
and 69
i 67
you 53
of 51
is 50
it 49
for 44
in 43
that 35
on 30
me 30
my 27
be 25
this 22
are 22
your 21
but 21


# задание 7
## Найти количество подписчиков (фолловеров) у авторов твитов и вывести топ-10 ( то есть самое большое кол-во подписчиков и дальше по убыванию)
### для каждого твита добавляю кол-во подписчиков внутрь множества, потом множество преобразовываю в список, сортирую его по убыванию

In [12]:
followers = set()
for tw in twitter:
    if tw.get('user'):
        followers.add(tw['user']['followers_count'])
n_followers = list(followers)
followersort = sorted(n_followers, reverse=True)
print('ОТВЕТ:')
print('top 10: САМОЕ БОЛЬШОЕ КОЛ-ВО ПОДПИСЧИКОВ:')
print()
for i in range(10):
    print(followersort[i])

ОТВЕТ:
top 10: САМОЕ БОЛЬШОЕ КОЛ-ВО ПОДПИСЧИКОВ:

2521403
1491309
1206759
1137374
625463
392472
383698
374222
318189
311319


# задание 8
## Топ-10 источников твита (из какого приложения написан) (подсказка: для обработки используйте регулярные выражения)
### по ключу source в значении лежит строка, из нее достаю название ресурса с помощью рег. выражений
### потом делаю частотный список

In [13]:
# 8 Топ-10 источников твита (из какого приложения написан) regexp
sources = []
for tw in twitter:
    if tw.get('source'):
        pat = '>([^<]*)<'
        res = re.search(pat, tw['source']).group(1)
        sources.append(res)
sourcesort = Counter(sources)

print('ОТВЕТ:')
print('top 10: САМЫЕ ЧАСТОТНЫЕ ИСТОЧНИКИ:')
print()
for key, val in sourcesort.most_common(10):
    print(key, val)

ОТВЕТ:
top 10: САМЫЕ ЧАСТОТНЫЕ ИСТОЧНИКИ:

Twitter for iPhone 800
Twitter for Android 695
Twitter Web Client 140
twittbot.net 122
Twitter Lite 51
Twitter for iPad 28
TweetDeck 23
Facebook 17
IFTTT 14
تطبيق قرآني 10


# СПАСИБО ЗА ВНИМАНИЕ!
### пожалуйста, в следующий раз пишите более точные формулировки заданий